In [2]:
%run 'template.ipynb'

In [3]:
df = pd.read_csv("flowers.csv")
df = df.replace({np.nan: None})
df

,Unnamed: 0,old_name,new_name
0,5d58b28f-9b31-4b4a-975f-d2c065a39330,Meadow brome,None
1,9ec5a11c-5a42-4af6-9af5-47beb0c7f169,Easter-bell,Весенняя колокольчик
2,aa126c55-99ed-429f-b1ea-7204f01a21ed,Field speedwell,Полынь полевая
3,eeb6a4f7-fbd3-44a3-807b-56a5bde966b4,Umbrella pine,None
4,1da23ba2-886e-4cf8-b023-30e6ae63fe2e,White cockle,Белая коклюшка
...,...,...,...
95,a62c8b52-8ac4-421c-8124-a3cfcf5a7dbb,Bitter dock,Лапчатка горькая
96,674f956c-fee9-43cd-9f7a-a7a850b4e45f,Oxeye daisy,Льнянка
97,cf76c942-6de2-40fe-83f3-a117143f272a,Male fern,Мужской ферн
98,480990f0-9541-4fb9-a643-3b32b5fc2231,Felonherb,None


In [6]:
df.set_index("Unnamed: 0", inplace=True)
df

,old_name,new_name
Unnamed: 0,,
5d58b28f-9b31-4b4a-975f-d2c065a39330,Meadow brome,None
9ec5a11c-5a42-4af6-9af5-47beb0c7f169,Easter-bell,Весенняя колокольчик
aa126c55-99ed-429f-b1ea-7204f01a21ed,Field speedwell,Полынь полевая
eeb6a4f7-fbd3-44a3-807b-56a5bde966b4,Umbrella pine,None
1da23ba2-886e-4cf8-b023-30e6ae63fe2e,White cockle,Белая коклюшка
...,...,...
a62c8b52-8ac4-421c-8124-a3cfcf5a7dbb,Bitter dock,Лапчатка горькая
674f956c-fee9-43cd-9f7a-a7a850b4e45f,Oxeye daisy,Льнянка
cf76c942-6de2-40fe-83f3-a117143f272a,Male fern,Мужской ферн


In [10]:
df.loc["5d58b28f-9b31-4b4a-975f-d2c065a39330"]

old_name    Meadow brome
new_name            None
Name: 5d58b28f-9b31-4b4a-975f-d2c065a39330, dtype: object

In [4]:
from sqlmodel import SQLModel, Field
from uuid import UUID, uuid4
from datetime import date


class Plant(SQLModel, table=True):
    __tablename__ = "plants"
    __table_args__ = {"extend_existing": True}
    id: UUID = Field(primary_key=True, default_factory=uuid4)
    name: str | None
    origin_name: str | None
    bibliography: str
    slug: str
    year: date
    image_url: str
    family: str
    genus: str
    rank: str

In [5]:
from sqlmodel import select

async with session_maker() as session:
    plants = (await session.exec(select(Plant))).all()

In [ ]:
async with session_maker() as session:
    for plant in plants:
        plant.name = df.loc[str(plant.id)]["new_name"]
        session.add(plant)

    await session.commit()